In [172]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis 
pyLDAvis.enable_notebook()
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [173]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
import string
import gensim 
from gensim import corpora
import nltk
#https://www.zhihu.com/question/34801598
#https://zhuanlan.zhihu.com/p/62414576
# %load 11.py  
#https://radimrehurek.com/gensim/models/ldamodel.html
#https://toutiao.io/posts/c7853u/preview
#https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb
#https://nbviewer.jupyter.org/github/bmabey/hacker_news_topic_modelling/blob/master/HN%20Topic%20Model%20Talk.ipynb#topic=0&lambda=1&term=

In [174]:
pwd

'/Users/yuming/Desktop/Project/lingyu/sscm'

In [175]:
sgcm = pd.read_excel('./data.xlsx',sheet_name="sgcm")
scm = pd.read_excel('./data.xlsx',sheet_name="scm")
gcm = pd.read_excel('./data.xlsx',sheet_name="gscm")

#  clean data

In [176]:
#stemmer=SnowballStemmer("english", ignore_stopwords=True)
# stemmer.stem("runing")

In [177]:
stop = set(stopwords.words('english')) 
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [178]:
#icis.head(1)

In [179]:
abstract_na_sgcm=sgcm.Definition.dropna()
abstract_na_scm=scm.Definition.dropna()
abstract_na_gcm=gcm.Definition.dropna()

In [180]:
doc_complete_sgcm =abstract_na_sgcm
doc_complete_scm =abstract_na_scm
doc_complete_gcm =abstract_na_gcm

In [181]:
def clean(doc):     
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])     
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())     
    return normalized  
doc_clean_sgcm = [clean(doc).split() for doc in doc_complete_sgcm]
doc_clean_scm = [clean(doc).split() for doc in doc_complete_scm]
doc_clean_gcm = [clean(doc).split() for doc in doc_complete_gcm]

# prepare doc-term matrix

In [230]:
dictionary_sgcm = corpora.Dictionary(doc_clean_sgcm) #build index for each word
dictionary_scm = corpora.Dictionary(doc_clean_scm)
dictionary_gcm = corpora.Dictionary(doc_clean_gcm)


In [231]:
# assume the word 'b' is to be deleted, put its id in a variable
del_ids_sgcm = [k for k,v in dictionary_sgcm.items() if v=='supply' or v=='chain' or v=='management']
del_ids_scm = [k for k,v in dictionary_scm.items() if v=='supply' or v=='chain' or v=='management']
del_ids_gcm = [k for k,v in dictionary_gcm.items() if v=='supply' or v=='chain' or v=='management']

In [232]:
# remove unwanted word ids from the dictionary in place
dictionary_sgcm.filter_tokens(bad_ids=del_ids_sgcm)
dictionary_scm.filter_tokens(bad_ids=del_ids_scm)
dictionary_gcm.filter_tokens(bad_ids=del_ids_gcm)

In [233]:
#dictionary.filter_extremes(no_below=20, no_above=0.5)  # Filter out words that occur less than 20 documents, or more than 50% of the documents.
#dictionary.compactify()
doc_term_matrix_sgcm = [dictionary_sgcm.doc2bow(doc) for doc in doc_clean_sgcm] #build dt matrix
doc_term_matrix_scm = [dictionary_scm.doc2bow(doc) for doc in doc_clean_scm]
doc_term_matrix_gcm = [dictionary_gcm.doc2bow(doc) for doc in doc_clean_gcm]

In [234]:
#Let’s see how many tokens and documents we have to train on.
print('Number of unique tokens: %d' % len(dictionary)) 
print('Number of documents: %d' % len(doc_term_matrix))

Number of unique tokens: 306
Number of documents: 35


In [235]:
#print(dictionary.token2id)

# build LDA

In [236]:
Lda_sgcm = gensim.models.ldamodel.LdaModel
Lda_scm = gensim.models.ldamodel.LdaModel
Lda_gcm = gensim.models.ldamodel.LdaModel

In [237]:
ldamodel_sgcm = Lda_sgcm(doc_term_matrix_sgcm,id2word = dictionary_sgcm, chunksize=2000,     
               alpha='auto',     
               eta='auto',     
               iterations=400,     
               num_topics=7,     
               passes=50,     
               eval_every=1)
ldamodel_scm = Lda_scm(doc_term_matrix_scm,id2word = dictionary_scm, chunksize=2000,     
               alpha='auto',     
               eta='auto',     
               iterations=400,     
               num_topics=7,     
               passes=50,     
               eval_every=1)
ldamodel_gcm = Lda_gcm(doc_term_matrix_gcm,id2word = dictionary_gcm, chunksize=2000,     
               alpha='auto',     
               eta='auto',     
               iterations=400,     
               num_topics=7,     
               passes=50,     
               eval_every=1)

In [238]:
#dump and load

In [239]:
import json
with open("./doc_term_matrix_sgcm.txt", "w") as fp:
     json.dump(doc_term_matrix_sgcm, fp)
with open("./doc_term_matrix_scm.txt", "w") as fp:
     json.dump(doc_term_matrix_scm, fp)
with open("./doc_term_matrix_gcm.txt", "w") as fp:
     json.dump(doc_term_matrix_gcm, fp)


In [240]:
with open("./doc_term_matrix_sgcm.txt", "r") as fp:
     doc_term_matrix_sgcm = json.load(fp)
with open("./doc_term_matrix_scm.txt", "r") as fp:
     doc_term_matrix_scm = json.load(fp)
with open("./doc_term_matrix_gcm.txt", "r") as fp:
     doc_term_matrix_gcm = json.load(fp)

In [241]:
dictionary_sgcm.save('./dictionary_sgcm')
dictionary_scm.save('./dictionary_scm')
dictionary_gcm.save('./dictionary_gcm')
dictionary_sgcm=gensim.corpora.dictionary.Dictionary.load('./dictionary_sgcm')
dictionary_scm=gensim.corpora.dictionary.Dictionary.load('./dictionary_scm')
dictionary_gcm=gensim.corpora.dictionary.Dictionary.load('./dictionary_gcm')

In [242]:
ldamodel_sgcm.save("./ldamodel_sgcm-7")
ldamodel_scm.save("./ldamodel_scm-7")
ldamodel_gcm.save("./ldamodel_gcm-7")

In [243]:
ldamodel_sgcm_7 = Lda.load("./ldamodel_sgcm-7")
ldamodel_scm_7 = Lda.load("./ldamodel_scm-7")
ldamodel_gcm_7 = Lda.load("./ldamodel_gcm-7")

In [244]:
ldamodel_sgcm_7.show_topic(0, topn=10) # show specific topic with keyword

[('zero', 0.036885627),
 ('environment', 0.029713394),
 ('product', 0.029713383),
 ('green', 0.022541158),
 ('waste', 0.022541158),
 ('management', 0.0153689375),
 ('protection', 0.015368924),
 ('improve', 0.015368915),
 ('system', 0.015368913),
 ('characterized', 0.008196664)]

In [245]:
ldamodel_sgcm_7.print_topics(num_topics=7, num_words=8)

[(0,
  '0.037*"zero" + 0.030*"environment" + 0.030*"product" + 0.023*"green" + 0.023*"waste" + 0.015*"management" + 0.015*"protection" + 0.015*"improve"'),
 (1,
  '0.067*"environmental" + 0.030*"performance" + 0.021*"customer" + 0.021*"management" + 0.021*"dimension" + 0.011*"development" + 0.011*"material" + 0.011*"goal"'),
 (2,
  '0.026*"material" + 0.014*"production" + 0.014*"refers" + 0.014*"inventory" + 0.014*"waste" + 0.014*"raw" + 0.014*"recovery" + 0.014*"product"'),
 (3,
  '0.031*"sustainable" + 0.023*"risk" + 0.016*"environmental" + 0.016*"market" + 0.016*"management" + 0.016*"taken" + 0.016*"development" + 0.016*"product"'),
 (4,
  '0.048*"management" + 0.043*"product" + 0.038*"environmental" + 0.024*"process" + 0.020*"sourcing" + 0.020*"material" + 0.020*"manufacturing" + 0.015*"endoflife"'),
 (5,
  '0.029*"green" + 0.029*"þ" + 0.029*"logistics" + 0.029*"reverse" + 0.020*"environmental" + 0.020*"concern" + 0.020*"management" + 0.010*"distribution"'),
 (6,
  '0.033*"environm

In [246]:
for doc in ldamodel_sgcm_7.get_document_topics(doc_term_matrix_sgcm,minimum_probability=0.01,per_word_topics=False):
    print(doc)

[(0, 0.99610776)]
[(1, 0.9940113)]
[(4, 0.9936221)]
[(3, 0.9935304)]
[(3, 0.9860063)]
[(4, 0.9879164)]
[(6, 0.9883182)]
[(5, 0.9941411)]
[(4, 0.99604803)]
[(6, 0.987359)]
[(4, 0.9879163)]
[(3, 0.9961068)]
[(4, 0.9739146)]
[(6, 0.9923708)]
[(3, 0.9903312)]
[(5, 0.98740566)]
[(5, 0.97517425)]
[(4, 0.9941726)]
[(1, 0.98156226)]
[(0, 0.99424595)]
[(6, 0.9873589)]
[(1, 0.9893649)]
[(1, 0.97899365)]
[(4, 0.99330616)]
[(2, 0.9866738)]
[(4, 0.973915)]
[(2, 0.99424064)]
[(1, 0.9639104), (4, 0.010285704)]
[(4, 0.9939098)]
[(0, 0.99325293)]
[(5, 0.98328894)]
[(4, 0.992137)]
[(1, 0.572866), (4, 0.013522204), (6, 0.38727197)]
[(5, 0.9883613)]
[(0, 0.97468704)]


In [247]:
doc_topic_max = []  # the top topic in each doc
doc_topic_per=[]  # the likelihood of this top topic
for doc in ldamodel.get_document_topics(doc_term_matrix):     
    doc_topic = []     
    for topic in doc:         
        doc_topic.append(topic[1])     
    doc_topic_max.append(doc[doc_topic.index(max(doc_topic))][0]) 
    doc_topic_per.append(doc[doc_topic.index(max(doc_topic))][1])

In [248]:
ldamodel.get_document_topics(doc_term_matrix)

In [249]:
doc_topic_max

[6,
 5,
 5,
 2,
 4,
 4,
 6,
 6,
 2,
 5,
 6,
 6,
 4,
 0,
 0,
 0,
 1,
 5,
 2,
 3,
 0,
 4,
 1,
 5,
 5,
 5,
 1,
 4,
 0,
 6,
 1,
 0,
 5,
 1,
 6]

In [250]:
doc_topic_per

[0.9965213,
 0.9947164,
 0.9938046,
 0.9944218,
 0.9893578,
 0.98855025,
 0.9888658,
 0.9947928,
 0.995401,
 0.9898718,
 0.9888658,
 0.9965213,
 0.97086024,
 0.826942,
 0.9907041,
 0.98765093,
 0.98323107,
 0.99380463,
 0.98455656,
 0.99434286,
 0.9876509,
 0.9906735,
 0.983231,
 0.99352527,
 0.98820835,
 0.77935624,
 0.99490005,
 0.97557706,
 0.9932191,
 0.99366957,
 0.983231,
 0.9912453,
 0.972268,
 0.98832095,
 0.9820584]

# more things can do, like find the most typical document for each topic and the topics distributon and Numbers

# visualize topics

In [251]:
import warnings 
import pyLDAvis.gensim_models
warnings.filterwarnings('ignore') 
prepared_data_sgcm = pyLDAvis.gensim_models.prepare(ldamodel_sgcm_7,doc_term_matrix_sgcm, dictionary_sgcm)
prepared_data_scm = pyLDAvis.gensim_models.prepare(ldamodel_scm_7,doc_term_matrix_scm, dictionary_scm)
prepared_data_gcm = pyLDAvis.gensim_models.prepare(ldamodel_gcm_7,doc_term_matrix_gcm, dictionary_gcm)

In [252]:
prepared_data_sgcm

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.138996 -0.019930       1        1  29.090744
0     -0.099499 -0.121172       2        1  16.559025
3     -0.037839  0.147352       3        1  15.696463
5     -0.062079  0.048658       4        1  11.374881
1      0.061091 -0.018703       5        1  10.303519
6      0.043878 -0.013121       6        1  10.045993
2     -0.044549 -0.023083       7        1   6.929375, topic_info=              Term      Freq      Total Category  logprob  loglift
12   environmental  16.00000  16.000000  Default  30.0000  30.0000
236           zero   4.00000   4.000000  Default  29.0000  29.0000
206          green   4.00000   4.000000  Default  28.0000  28.0000
63         process   6.00000   6.000000  Default  27.0000  27.0000
62     performance   4.00000   4.000000  Default  26.0000  26.0000
..             ...       ...        ...      ...      ...      ...
128      logistics   0.55055   4.286161   Topic7  -4.2905   0.6172
235          waste   0.55055   3.176411   Topic7  -4.2905   0.9168
12   environmental   0.55055  16.484680   Topic7  -4.2905  -0.7299
15         improve   0.55055   3.066514   Topic7  -4.2905   0.9520
62     performance   0.55055   4.435324   Topic7  -4.2905   0.5830

[336 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
153       3  0.766045      ability
40        1  0.373329      account
40        3  0.373329      account
40        5  0.373329      account
57        1  0.703019  achievement
...     ...       ...          ...
78        3  0.766045        which
193       5  0.561667        whole
193       6  0.561667        whole
236       2  0.982232         zero
210       4  0.820721            þ

[382 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 6, 2, 7, 3])

In [253]:
prepared_data_scm

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.106723  0.084873       1        1  35.122788
3     -0.089884 -0.124417       2        1  27.794979
4      0.160111 -0.029659       3        1  18.672669
5      0.039915  0.059334       4        1   9.539698
2     -0.002128  0.024668       5        1   4.777564
1     -0.015866 -0.017192       6        1   4.044954
6      0.014574  0.002392       7        1   0.047347, topic_info=             Term      Freq     Total Category  logprob  loglift
12  environmental  6.000000  6.000000  Default  30.0000  30.0000
35         social  6.000000  6.000000  Default  29.0000  29.0000
9        economic  4.000000  4.000000  Default  28.0000  28.0000
63        process  3.000000  3.000000  Default  27.0000  27.0000
52     management  5.000000  5.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
25   organization  0.000691  1.252248   Topic7  -5.2311   0.1534
26    procurement  0.000691  1.990994   Topic7  -5.2311  -0.3103
27        product  0.000691  3.428320   Topic7  -5.2311  -0.8537
28     production  0.000691  2.074707   Topic7  -5.2311  -0.3515
29  profitability  0.000691  1.252248   Topic7  -5.2311   0.1534

[348 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
153       3  0.887447      ability
40        1  0.437868      account
40        2  0.437868      account
115       2  0.826699      achieve
57        1  0.798564  achievement
...     ...       ...          ...
55        2  0.665589        three
114       2  0.586966          use
114       4  0.586966          use
39        1  0.798564        volun
56        2  0.826699         well

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 6, 3, 2, 7])

In [254]:
prepared_data_gcm

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.048867  0.150915       1        1  23.494200
6      0.169426 -0.040184       2        1  20.455327
4      0.099921  0.013921       3        1  17.580878
5     -0.105930 -0.115665       4        1  14.921767
2     -0.063151 -0.010560       5        1  12.138759
1     -0.058507  0.002109       6        1   7.482917
3      0.007107 -0.000535       7        1   3.926153, topic_info=             Term       Freq      Total Category  logprob  loglift
42        product  10.000000  10.000000  Default  30.0000  30.0000
99    environment   3.000000   3.000000  Default  29.0000  29.0000
9   environmental  10.000000  10.000000  Default  28.0000  28.0000
78           zero   4.000000   4.000000  Default  27.0000  27.0000
61       practice   2.000000   2.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
15      packaging   0.048213   1.304824   Topic7  -5.5215  -0.0607
16      principle   0.048213   2.030734   Topic7  -5.5215  -0.5030
17    procurement   0.048213   1.304824   Topic7  -5.5215  -0.0607
18            set   0.048213   1.916540   Topic7  -5.5215  -0.4451
19          whole   0.048213   1.796589   Topic7  -5.5215  -0.3805

[303 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
20        4  0.519214   achieve
0         3  0.766387    across
1         1  0.711551  activity
1         3  0.355776  activity
137       2  0.748375       add
...     ...       ...       ...
57        2  0.748375      well
19        3  0.556610     whole
19        6  0.556610     whole
78        1  0.914249      zero
40        4  0.774049         þ

[212 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 5, 6, 3, 2, 4])

In [255]:
# 2d list to one d
from itertools import chain  
#words = chain.from_iterable(df.to_list())

In [256]:
words = chain.from_iterable(doc_clean)

In [257]:
from collections import Counter  
c = Counter(words)  
c.most_common(50)

[('chain', 24),
 ('environmental', 24),
 ('supply', 22),
 ('management', 22),
 ('product', 18),
 ('social', 9),
 ('process', 9),
 ('material', 8),
 ('manufacturing', 7),
 ('economic', 6),
 ('performance', 6),
 ('environment', 6),
 ('logistics', 6),
 ('green', 6),
 ('integration', 5),
 ('sustainable', 5),
 ('design', 5),
 ('sourcing', 5),
 ('activity', 5),
 ('zero', 5),
 ('production', 4),
 ('distribution', 4),
 ('improve', 4),
 ('company', 4),
 ('taken', 4),
 ('sustainability', 4),
 ('impact', 4),
 ('selection', 4),
 ('market', 4),
 ('resource', 4),
 ('including', 4),
 ('waste', 4),
 ('interorganizational', 3),
 ('business', 3),
 ('procurement', 3),
 ('order', 3),
 ('goal', 3),
 ('dimension', 3),
 ('development', 3),
 ('account', 3),
 ('customer', 3),
 ('set', 3),
 ('issue', 3),
 ('use', 3),
 ('life', 3),
 ('cycle', 3),
 ('achieve', 3),
 ('risk', 3),
 ('þ', 3),
 ('reverse', 3)]

In [258]:
#from nltk.probability import FreqDist 
#fdist = FreqDist(words) 
#fdist.most_common(20)

In [259]:
"ra" in words

False

In [260]:
words